In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
from torch import nn
from torch import optim
import torchvision
from torchvision import datasets, transforms, models
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import Dataset, DataLoader
from collections import OrderedDict
from PIL import Image
import seaborn as sns
import splitfolders
from IPython.display import display, HTML

import torchtext
from torchtext.data import get_tokenizer   # for tokenization
from collections import Counter     # for tokenizer


In [2]:
# splitfolders.ratio("Food Images", output="food_images", 
#                    seed=30, ratio=(.7, .2, .1), 
#                    group_prefix=None)

In [3]:
data_df = pd.read_csv('Food Ingredients and Recipe Dataset with Image Name Mapping.csv')
data_df.iloc[1,4]

'crispy-salt-and-pepper-potatoes-dan-kluger'

In [4]:
data_dir = './food_images'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/val'
test_dir = data_dir + '/test'

In [5]:
# img,recipe_tokens = torch_data[10]

# fig, ax = plt.subplots()
# img


In [6]:
# [torch_data.vocab.itos[token] for token in recipe_tokens.tolist()]

In [7]:
# image_data_ref = pd.read_csv('Food Ingredients and Recipe Dataset with Image Name Mapping.csv')
# image_data_ref.ndim
# display(image_data_ref)


In [8]:
##recipe

In [10]:
from dataloader import *
from model import *

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(256),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485,0.456,0.406],
                                                           [0.229,0.224,0.225])])
torch_data = CustomDataset(recipe_csv='Food Ingredients and Recipe Dataset with Image Name Mapping.csv',
                                    img_dir='Food Images', transform=train_transforms)

path = 'my_checkpoint.pth.tar'
checkpoint = torch.load(path)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Hyperparameters
embed_size = 256
hidden_size = 256
vocab_size = len(torch_data.vocab)
num_layers = 2
learning_rate = 3e-4
num_epochs = 1

model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
model.load_state_dict(checkpoint['state_dict'])

# No use
# optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
# epoch = checkpoint['epoch']  # ALREADY DEFFINED ABOE
# loss = checkpoint['loss']

C:\Users\erice\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\erice\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


<All keys matched successfully>

In [102]:
model.eval()
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(256),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485,0.456,0.406],
                                                           [0.229,0.224,0.225])])
test_img1 = train_transforms(Image.open("Food Images/12-minute-saucy-chicken-breasts-with-limes.jpg").convert("RGB")).unsqueeze(0)
teststr = model.recipe_generate(test_img1, torch_data.vocab, max_length = 500)
" ".join(teststr)



'<START> heat a small bowl . heat 1 tablespoon oil in a saucepan heat , then mix together and sugar , beat butter and sugar in a bowl and cook , covered , until combined . remove from heat and simmer , swirling internal heat , 1 minute . add dough and half of parsley in plates . add 1/4 cup oil and 1 tsp . add pecans and cook until reduced to boil , often , 20 minutes . pour half . rewarm rice and serve . <END>'

In [103]:
teststr.remove('<START>')
teststr

['heat',
 'a',
 'small',
 'bowl',
 '.',
 'heat',
 '1',
 'tablespoon',
 'oil',
 'in',
 'a',
 'saucepan',
 'heat',
 ',',
 'then',
 'mix',
 'together',
 'and',
 'sugar',
 ',',
 'beat',
 'butter',
 'and',
 'sugar',
 'in',
 'a',
 'bowl',
 'and',
 'cook',
 ',',
 'covered',
 ',',
 'until',
 'combined',
 '.',
 'remove',
 'from',
 'heat',
 'and',
 'simmer',
 ',',
 'swirling',
 'internal',
 'heat',
 ',',
 '1',
 'minute',
 '.',
 'add',
 'dough',
 'and',
 'half',
 'of',
 'parsley',
 'in',
 'plates',
 '.',
 'add',
 '1/4',
 'cup',
 'oil',
 'and',
 '1',
 'tsp',
 '.',
 'add',
 'pecans',
 'and',
 'cook',
 'until',
 'reduced',
 'to',
 'boil',
 ',',
 'often',
 ',',
 '20',
 'minutes',
 '.',
 'pour',
 'half',
 '.',
 'rewarm',
 'rice',
 'and',
 'serve',
 '.',
 '<END>']

In [104]:
refstr = 'Place a 10"" cast-iron skillet in oven; preheat oven to 475°F. Lay your hands on another heavy skillet or saucepan the same diameter or smaller and tear off an 8"" piece of foil; set aside near stove. Mix cumin, turmeric, paprika, and 1 tsp. salt in a small bowl. Brace yourself: This next step requires some light butchery, but it’s easy. Place chicken skin side down on a cutting board and find the short row of small rib bones near the outer edge of the breast. Grip them with your fingers and pull them off (or use a paring knife or shears). Next: along the center line where the breast was cut in half, there’s a dark, vaguely rectangular bone—it almost looks like an anvil. Sometimes it’s in one piece, or both sides will have a half of it. It doesn’t matter: Grip the bone with your fingers and pull it out, then tease out the strip of flexible cartilage just below it. All done! Now the chicken will lay flat in the pan, which is why it cooks so quickly.Season both sides of chicken with salt, then with all of the spice rub. Remove hot skillet from oven and set over medium-high heat. Add oil, then lay chicken into pan, skin side down. Use a spatula to firmly press chicken against surface, then add 1 lime to pan, placing sides down. Cook until skin is starting to brown, about 2 minutes. Transfer skillet to oven, lay reserved foil over chicken, then weight with second pan. Cook 10 minutes.Remove pan from oven and turn chicken over; cook second side in residual heat 1 minute. Squeeze juice from cooked limes into pan; transfer chicken and pan juices to a plate and serve with remaining lime halves.'


refstr_tok = torch_data.vocab.tokenizer(str(refstr))
print(refstr_tok)
len(refstr_tok)

['place', 'a', '10', 'cast-iron', 'skillet', 'in', 'oven', 'preheat', 'oven', 'to', '475°f', '.', 'lay', 'your', 'hands', 'on', 'another', 'heavy', 'skillet', 'or', 'saucepan', 'the', 'same', 'diameter', 'or', 'smaller', 'and', 'tear', 'off', 'an', '8', 'piece', 'of', 'foil', 'set', 'aside', 'near', 'stove', '.', 'mix', 'cumin', ',', 'turmeric', ',', 'paprika', ',', 'and', '1', 'tsp', '.', 'salt', 'in', 'a', 'small', 'bowl', '.', 'brace', 'yourself', 'this', 'next', 'step', 'requires', 'some', 'light', 'butchery', ',', 'but', 'it’s', 'easy', '.', 'place', 'chicken', 'skin', 'side', 'down', 'on', 'a', 'cutting', 'board', 'and', 'find', 'the', 'short', 'row', 'of', 'small', 'rib', 'bones', 'near', 'the', 'outer', 'edge', 'of', 'the', 'breast', '.', 'grip', 'them', 'with', 'your', 'fingers', 'and', 'pull', 'them', 'off', '(', 'or', 'use', 'a', 'paring', 'knife', 'or', 'shears', ')', '.', 'next', 'along', 'the', 'center', 'line', 'where', 'the', 'breast', 'was', 'cut', 'in', 'half', ',', '

337

In [105]:
from torchtext.data.metrics import bleu_score

bleu_score([teststr],[[refstr_tok]])

0.006742538396031864

In [6]:
from train import load_checkpoint
from dataloader import *
from model import *

train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(256),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485,0.456,0.406],
                                                           [0.229,0.224,0.225])])
# valid_transforms = transforms.Compose([transforms.Resize(255),
#                                     transforms.CenterCrop(size=224),
#                                     transforms.ToTensor(),
#                                     transforms.Normalize([0.485,0.456,0.406],
#                                                         [0.229,0.224,0.225])])
# test_transforms = transforms.Compose([transforms.Resize(255),
#                                     transforms.CenterCrop(size=224),
#                                     transforms.ToTensor(),
#                                     transforms.Normalize([0.485,0.456,0.406],
#                                                         [0.229,0.224,0.225])])

torch_data = CustomDataset(recipe_csv='Food Ingredients and Recipe Dataset with Image Name Mapping.csv',
                                img_dir='Food Images', transform=train_transforms)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Hyperparameters
embed_size = 256
hidden_size = 256
vocab_size = len(torch_data.vocab)
num_layers = 2
learning_rate = 3e-4
num_epochs = 5
model = CNNtoRNN(embed_size, hidden_size, vocab_size, num_layers).to(device)
criterion = nn.CrossEntropyLoss(ignore_index=torch_data.vocab.stoi["<PAD>"])
optimizer = optim.Adam(model.parameters(), lr=.003)
load_checkpoint(torch.load("my_checkpoint.pth.tar"), model, optimizer)

C:\Users\erice\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\erice\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet101_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet101_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Loading checkpoint ... 


2110

In [ ]:
model.recipe_generate()